In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import stumpy
import glob
import ipywidgets as widgets
import peakutils
from tslearn.clustering import KShape
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
from tslearn.clustering import KernelKMeans
from tslearn.clustering import TimeSeriesKMeans
from scipy.signal import savgol_filter
from functools import partial
from joblib import delayed, Parallel
%matplotlib widget

/share/python-virtual-environments/daniel-jl/lib/python3.6/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.cluster.k_means_ module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.cluster. Anything that cannot be imported from sklearn.cluster is now part of the private API.
  warnings.warn(message, FutureWarning)


# Matrix Profile Motif Detection

In this notebook we use the matrix profile to scrape motifs from all timeseries, and compile them into a dataset of shape `(n_motifs, len_motifs, n_dimensions)`, where `n_motifs` is the number of motifs scraped from all data, `len_motifs` is the chosen window size and `n_dimensions` is 7, for the 7 curvature values we will be considering. 

First we will load the raw data:

In [2]:
path = "/share/data/temp/athira/July17_features_combined_noLightStimuli.pickle"
df = pd.read_pickle(path)

Then we need to define some of the functions that will do our work. First a function that takes a filename/unique id and slices the timeseries for that id out of the main dataframe:

In [3]:
def get_curvature_data(f):
    """
    function to extract a specific timeseries from the main dataframe.
    """
    df_f = df[df["filename"] == f]
    cols_to_return = [c for c in df.columns if "curv" in c]
    return df_f[cols_to_return]

Then a function that takes a timeseries and a window size, calculates the matrix profile, and returns the motifs it finds at the minima of the matrix profile.

In [4]:
def get_motifs(data, window):
    """
    Uses the matrix profile to scrape motifs from a timeseries. Returns motifs in the form (n_motifs, motif_length, n_dim).
    """
    try:
        mp, ind = stumpy.mstump(data, m = window)
        peaks = peakutils.indexes(1-mp[:,-1], min_dist = window, thres = 0.8)
        motifs = np.stack([data[peak:peak+window, :] for peak in peaks])
        return motifs  
    except Exception as e:
        print(e)

Last a function that we can use to do our work efficiently using joblib.

In [9]:
def plot_motifs(curvs, peaks, mp, window = 150):
    fig, axes = plt.subplots(8,1, figsize = (10,6), sharex=True)
    for i, ax in enumerate(axes):
        try:
            ax.plot(curvs[:,i], lw = 0.5)
        except:
            pass
    axes[-1].plot(mp[:,-1], lw = 0.5)
    axes[-1].scatter(peaks, mp[peaks,-1], c = "r", s = 1)
    axes[-1].set_ylabel("MP")
    axes[-1].set_xlabel("Time (frames)")
    axes[-1].set_yticks([])
    for peak in peaks:
        for i in range(7):
            axes[i].plot(np.arange(peak, peak+window), curvs[peak:peak+window, i])
            axes[i].set_ylabel(f"Curv {i}")
            axes[i].set_yticks([])
    return fig
    

In [39]:
def do_work(filename, window = 150):
    try:
        data = get_curvature_data(filename)
        for c in data.columns:
            data[c] = data[c].rolling(10).mean()
        data = data[10:]
        mp, ind = stumpy.mstump(data, m = window)
        peaks = peakutils.indexes(1-mp[:,-1], min_dist = window, thres = 0.8)
        motifs = np.stack([data.values[peak:peak+window, :] for peak in peaks])
        df = pd.DataFrame(zip(mp, peaks), columns = ["motifs", "motif_start"])
        df["filename"] = filename
    
        return df, data, mp, peaks
    except Exception as e:
        return( (filename, str(e)))

Then we gather all unique filenames that we want to scrape motifs for:

In [40]:
to_do = df["filename"].unique()

Set the windows we want to find motifs over:

In [ ]:

numbers = np.random.choice(range(len(to_do)), 25)
for number in numbers:
    for window in [30,150]:
        test = do_work(to_do[number], window)
        fig, ax = plt.subplots(8,1, figsize = (10,6), sharex = True)

        motifs, curves, mp, peaks = test
        ax[-1].plot(mp[:,-1])
        ax[-1].scatter(peaks, mp[peaks,-1], s = 3, c = "r")
        ax[-1].set_ylabel("MP")
        ax[-1].set_xlabel("Time (frames)")
        ax[-1].set_yticks([])

        for i, c in enumerate(curves.columns):
            ax[i].plot(curves[c])

        curves = curves.values
        for peak in peaks:
                for i in range(7):
                    ax[i].plot(np.arange(peak, peak+window), curves[peak:peak+window, i])
                    ax[i].set_ylabel(f"Curve {i}")
                    ax[i].set_yticks([])

        fig.savefig(f"./traces/{number}_{window}.png")
        fig.savefig(f"./traces/{number}_{window}.svg")

In [65]:
peaks

array([1495, 1676, 2601, 3147])

In [31]:
pd.DataFrame(zip(m, p), columns = ["motifs", "frame"])

,motifs,frame
0,"[[0.001816929379128851, -0.005239114654250443,...",658
1,"[[0.000689667479309719, -0.009880717378109694,...",811
2,"[[-0.00038045951368985697, -0.0033348528377246...",1830
3,"[[0.003582046739757061, -0.005616012160317041,...",3058
4,"[[-0.00016893554711714387, 0.00360996909439563...",3259
5,"[[0.00022864136553835124, -0.00841008488787338...",4778
6,"[[-0.0010266736266203226, -0.00335809504613280...",6009
7,"[[-0.0003301058444776572, -0.00882316405186429...",6917
8,"[[-0.0003341469215229154, -0.01094033768167719...",8018
9,"[[-0.008177888067439198, -0.006341102626174688...",8715


In [9]:
p.shape

(10,)

In [17]:
np.vstack([m, p])

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 3 dimension(s) and the array at index 1 has 2 dimension(s)

In [75]:
d = get_curvature_data(np.random.choice(to_do))

In [76]:
cols = d.columns
for c in cols:
    d[f"{c}_f"] = d[c].rolling(5).mean()

In [77]:
d[c].rolling(10).mean().values

array([        nan,         nan,         nan, ..., -0.02004233,
       -0.02021583, -0.02046526])

In [78]:
fig, ax = plt.subplots(7,1)
for i, c in enumerate(cols):
    ax[i].plot(d[c])
    ax[i].plot(d[f"{c}_f"])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [39]:
motifs = [do_work(x, window = 150) for x in np.random.choice(to_do, 2)]

In [41]:
pd.concat(motifs)

,motifs,motif_start,filename
0,"[[0.001816929379128851, -0.005239114654250443,...",658,20180730_112620_1_5m0s_Octopamine_None_None_sk...
1,"[[0.000689667479309719, -0.009880717378109694,...",811,20180730_112620_1_5m0s_Octopamine_None_None_sk...
2,"[[-0.00038045951368985697, -0.0033348528377246...",1830,20180730_112620_1_5m0s_Octopamine_None_None_sk...
3,"[[0.003582046739757061, -0.005616012160317041,...",3058,20180730_112620_1_5m0s_Octopamine_None_None_sk...
4,"[[-0.00016893554711714387, 0.00360996909439563...",3259,20180730_112620_1_5m0s_Octopamine_None_None_sk...
5,"[[0.00022864136553835124, -0.00841008488787338...",4778,20180730_112620_1_5m0s_Octopamine_None_None_sk...
6,"[[-0.0010266736266203226, -0.00335809504613280...",6009,20180730_112620_1_5m0s_Octopamine_None_None_sk...
7,"[[-0.0003301058444776572, -0.00882316405186429...",6917,20180730_112620_1_5m0s_Octopamine_None_None_sk...
8,"[[-0.0003341469215229154, -0.01094033768167719...",8018,20180730_112620_1_5m0s_Octopamine_None_None_sk...
9,"[[-0.008177888067439198, -0.006341102626174688...",8715,20180730_112620_1_5m0s_Octopamine_None_None_sk...


In [46]:
windows = [150]

And do the actual work. This will take about 1.5 hours per window on 20 threads. 

In [ ]:
for window in windows:
    #20 threads use up about 90gb of RAM. Don`t use much more.
    motifs = Parallel(n_jobs=20, verbose = 5)(delayed(partial(do_work, window = window))(f) for f in to_do)
    motifs = [x for x in motifs if type(x) != type(("this", "thing"))]
    motifs = pd.concat(motifs)
#     motifs = TimeSeriesScalerMeanVariance().fit_transform(motifs)
    motifs.to_hdf(f"motifs_window{window}_in_context.hdf5", key = "data")

[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  32 tasks      | elapsed:  1.7min
[Parallel(n_jobs=20)]: Done 122 tasks      | elapsed:  5.8min
[Parallel(n_jobs=20)]: Done 248 tasks      | elapsed: 10.9min
